In [1]:
import header
from header import __root__
from src import gs

🔑 Found password in password.txt (DEBUG MODE)
✅ Successfully opened KeePass database: C:\Users\user\Documents\repos\hypotez\secrets\credentials.kdbx
Failed to load GAPI credentials


2025-05-09 14:26:29,384 - DEBUG - 🐛 Нет нового релиза: https://github.com/repos/hypotez/hypo/releases/latest
 response.status_code=404 


In [2]:
import pandas as pd
import zipfile
from pathlib import Path
from typing import Optional, List, Dict, Any
from types import SimpleNamespace
from src.utils.jjson import j_loads, j_loads_ns
from src.utils.file import read_text_file, recursively_yield_file_path

In [7]:
class Config:
    """Класс конфигурации скрипта."""
    ENDPOINT: Path = __root__ / 'SANDBOX' / 'davidka'
    config: SimpleNamespace = j_loads_ns(ENDPOINT / 'davidka.json')
    STORAGE: Path
    try:
        STORAGE = Path(config.storage)
    except Exception as ex:
        raise ValueError(f"Ошибка загрузки пути к хранилищу: ", ex)

    GEMINI_API_KEY: Optional[str] = gs.credentials.gemini.onela.api_key # Установить имя владельца ключа
    GEMINI_MODEL_NAME: str = 'gemini-2.0-flash-exp' # Актуальное имя модели может отличаться, например, 'gemini-1.5-flash-latest'
    system_instructuction: str | None = read_text_file(ENDPOINT / 'instructions/analize_html.md')
    updated_links_file_name: str = 'updated_links.json'
    DELAY_AFTER_LINK_PROCESSING: int = 15
    WINDOW_MODE: str = 'headless'
    TRAIN_STORAGE:Path = Path(config.train_storage)
    unzip_path:Path = Path(config.unzipeped_storage)

In [ ]:
with zipfile.ZipFile("your_file.zip", 'r') as zip_ref:
    zip_ref.extractall(ENDPOINT / 'extracted_data')  # например, 'data'

In [11]:
#print(train_files)
products_dict:dict = {}
for file in recursively_yield_file_path(Config.TRAIN_STORAGE,'*.json'):
    file_dict = j_loads(file)
    products_dict.update(file_dict)


In [12]:
print(len(products_dict))


3728


In [13]:
df = pd.DataFrame(products_dict)

In [ ]:
print(df.head(100))

                                       https://additel.com/product.html/  \
html                   <div class="sticky"> <div class="w-full" id="f...   
text                      Follow Us\nServices\nCompany\nHave A Question?   
title                                                           Products   
meta_og_title                                                              
meta_name_title                                                            
meta_keywords          Dry Well Calibrator, Temperature Calibrator, D...   
meta_description       Additel designs and manufactures digital press...   
title_tag_content                                     Additel | Products   
supplier                                                     additel.com   
price                                                                      
specification                                                         []   
sku                                                                        
summary     

In [15]:
print(df.columns)

Index(['https://additel.com/product.html/',
       'https://www.adalet.com/products/dust-ignition-proof-enclosures-stainless-steel',
       'https://actisense.com/acti_software/ebl-reader/',
       'https://actisense.com/acti_software/nmea-reader/',
       'https://actisense.com/acti_software/toolkit/',
       'https://actisense.com/acti_software/sdk/',
       'https://www.acrartex.com/products/bivy-stick/',
       'https://www.acrartex.com/products/globalfix-v4-epirb/',
       'https://acim.nidec.com/aerospace',
       'https://acim.nidec.com/drives/control-techniques/products/ac-drives/powerdrive-f300',
       ...
       'https://www.wieland-electric.com/en/products/safety-technology/magnetic-safety-switches/',
       'https://www.wieland-electric.com/de/produkte/verbindungstechnik/industriesteckverbinder/revos-power/',
       'https://www.wieland-electric.com/de/produkte/sicherheitstechnik/sicherheitsschalter-mit-getrenntem-betaetiger/',
       'https://www.wieland-electric.com/en/p

In [30]:
df_transposed = df.T


In [31]:
print(df_transposed.columns)

Index(['html', 'text', 'title', 'meta_og_title', 'meta_name_title',
       'meta_keywords', 'meta_description', 'title_tag_content', 'supplier',
       'price', 'specification', 'sku', 'summary', 'description', 'page_type',
       'ai_analized_content', 'category', 'parent_category',
       'original_internal_url'],
      dtype='object')


In [22]:
df_clean = df_transposed.dropna(subset=['supplier'])


In [23]:
supplier_list = df_clean['supplier'].dropna().unique()
print(supplier_list)

['additel.com' 'adalet.com' 'actisense.com' 'acrartex.com'
 'acim.nidec.com' 'acehardware.com' 'abbeyextrusions.com' '50hz.com'
 '3mnz.co.nz' '3m.com' '3ds.com' '220.ge' '11st.co.kr' 'americanas.com.br'
 'aloktools.com' 'alliedtelesis.com' 'alliedmotion.com' 'allegromicro.com'
 'alfazalengineering.com' 'alfaelectric.com' 'aisinworld.com' 'airtac.com'
 'aimtti.com' 'aim-sportline.com' 'aifittings.com' 'agro.ch'
 'agriculture.hunterirrigation.com' 'aftermarket.tiautomotive.com'
 'aerospace.honeywell.com' 'aemc.com' 'advantest.com' 'batronix.com'
 'ballard.com' 'bartecusa.com' 'baldor.com' 'b2b.harting.com' 'axyz.com'
 'avtechtyee.com' 'aws-hydro.com' 'autonomylogic.com' 'autodesk.com'
 'audible.com' 'automatedlogic.com' 'armsol.in' 'aristaipc.com'
 'aspenavionics.com' 'appliedavionics.com' 'arduino.cc' 'apricus.com'
 'appareo.com' 'basiccopper.com' 'barksdale.com' 'balmar.net'
 'amphenolpcd.com' 'autronicafire.com' 'autel.com' 'autocom.se'
 'axiomtek.com' 'aviatar.com' 'aveva.com' 'autom

In [20]:
supplier_counts = df_clean.groupby('supplier').count().reset_index()

In [21]:
print(supplier_counts)

         supplier  html  text  title  meta_og_title  meta_name_title  \
0      11st.co.kr     3     3      3              3                3   
1          220.ge     3     3      3              3                3   
2         3ds.com     1     1      1              1                1   
3          3m.com     2     2      2              2                2   
4      3mnz.co.nz     1     1      1              1                1   
..            ...   ...   ...    ...            ...              ...   
773      xbox.com     4     4      4              4                4   
774     xecro.com     7     7      7              7                7   
775     xuron.com     5     5      5              5                5   
776     xylem.com     1     1      1              1                1   
777  yokogawa.com     4     4      4              4                4   

     meta_keywords  meta_description  title_tag_content  price  specification  \
0                3                 3                  

In [33]:
columns_to_check = ['parent_category', 'category']
existing_columns = [col for col in columns_to_check if col in df_transposed.columns]

# Если колонки существуют, считаем товары
if existing_columns:
    # Объединяем значения из существующих колонок
    category_counts = pd.concat([df[col] for col in existing_columns])

    # Считаем количество товаров в каждой категории
    category_counts = category_counts.dropna().value_counts().reset_index()
    category_counts.columns = ['category', 'product_count']
else:
    category_counts = pd.DataFrame(columns=['category', 'product_count'])

print(category_counts)

                   category  product_count
0                                     7451
1  Ultrafiltration Membrane              1
2          Metal Nameplates              1
3              Toiletry Bag              1
4          Bath Thermostats              1
5           Terminal Blocks              1
